In [1]:
"""
# Description:  Converts the U.S. Department of Interior's National 
#               Fire Plan Operations and Reporting System (NFPORS) dataset 
#               into the Task Force standardized schema.  Dataset
#               is enriched with vegetation, ownership, county, WUI, 
#               Task Force Region, and year.
#               BLM and NPS current year treatments are inlcuded but past 
#               years treatments excluded and managed by seperate notebooks.
# Author: Spatial Informatics Group LLC
# Version: 1.0.0
# Date Created: Jan 24, 2024
"""
import os
import arcpy
import datetime
from scripts.utils import init_gdb, og_file_input
from scripts._3a_NFPORS import Model_NFPORS_poly, Model_NFPORS_point

workspace, scratch_workspace = init_gdb()
arcpy.env.overwriteOutput = True

date_id = datetime.datetime.now().strftime("%Y-%m-%d").replace('-','') # like 20221216

# INPUTS
# Will need to re-download NFPORS dataset each for each update from the below web service URL
# TODO integrate Download Service script tool for NFPORS because the service is throttled to limit the number of downloadable features per call 
# https://usgs.nfpors.gov/arcgis/rest/services/nfpors_WM/MapServer/15

# BLM and NPS features are processed with a different tool from their own feature services

# finds the latest version by date (like 20240124 (YYYMMDD))
nfpors_poly = og_file_input(prefix = 'nfpors_fuels_treatments_', filetype = 'Polygon', gdb = os.path.join(workspace, "b_Originals"))
input_original_polys = os.path.join(workspace,'b_Originals',nfpors_poly)

# Downloads and saves BIA and FWS current year project points to scratch gdb prior to being appended together in Model_NFPORS
query_CA = "statename='California'"

bia_api = 'https://usgs.nfpors.gov/arcgis/rest/services/nfpors_WM/MapServer/1'
bia_fs = arcpy.FeatureSet(bia_api, query_CA)
input_original_pts_BIA = os.path.join(workspace,'b_Originals', f'NFPORSCurrentFYTreatmentsBIA_{date_id}')
get_bia_pt = arcpy.CopyFeatures_management(bia_fs, input_original_pts_BIA)
print("BIA current year points copied from rest service")

fws_api = 'https://usgs.nfpors.gov/arcgis/rest/services/nfpors_WM/MapServer/5'
fws_fs = arcpy.FeatureSet(fws_api, query_CA)
input_original_pts_FWS = os.path.join(workspace,'b_Originals', f'NFPORSCurrentFYTreatmentsFWS_{date_id}')
get_fws_pt = arcpy.CopyFeatures_management(fws_fs, input_original_pts_FWS)
print("FWS current year points copied from rest service")

# OUTPUTS
output_pts_enriched = os.path.join(workspace, 'c_Enriched',f'nfpors_fuels_treatments_pts_enriched_{date_id}')
output_polys_enriched = os.path.join(workspace,'c_Enriched',f'nfpors_fuels_treatments_polys_enriched_{date_id}')

Start Time 2024-02-14 17:25:13.804541
BIA current year points copied from rest service
FWS current year points copied from rest service


In [3]:
Model_NFPORS_poly(
    input_original_polys,
    output_polys_enriched, 
    output_pts_enriched
)


Part 1 Performing Polygons Standardization
   step 1/7 select by DOI agency (BIA, FWS, NPS)
   step 2/7 select after 1995
   step 3/7 select CA
   step 4/7 repair geometry
   step 5/7 rename and add fields
      assigning domains
   step 6/7 import attributes
          removing unnecessary fields
   Standardization Complete
     standardized has 8 records
   Executing Polygon Enrichments...
     Polygon Enrichment Start Time 2024-02-14 17:25:13.819195
     Calculating Broad Vegetation Type...
       enrich step 1/32 summarize veg within polygons
       enrich step 2/32 summarize attributes
       enrich step 3/32 add join
       enrich step 4/32 convert table to table
       enrich step 5/32 delete identical
         step has 8 records
       enrich step 6/32 add join
       enrich step 7/32 select layer by attribute
       enrich step 8/32 calculate user defined veg field yes
       enrich step 9/32 select layer by attribute
       enrich step 10/32 calculate veg domain code
       en

In [4]:
Model_NFPORS_point(
    input_original_pts_BIA, 
    input_original_pts_FWS, 
    output_pts_enriched
)

Part 2 Points Standardization
   step 1/6 select BIA points in CA
BIA Selected points has 108 records
   step 2/6 select FWS points in CA
FWS Selected points has 64 records
   step 3/6 combine points layers
appended points has 172 records
   step 4/6 rename and add fields
      assigning domains
   step 5/6 import attributes
          removing unnecessary fields
Standardization Complete
     standardized points has 172 records
   Executing Point Enrichments...
     Point Enrichment Start Time 2024-02-14 17:25:13.807480
     Calculating WUI...
       enrich step 1/16 select layer by WUI
       enrich step 2/16 calculate WUI yes
       enrich step 3/16 switch selection
       enrich step 4/16 calculate WUI no
       enrich step 5/16 clear selection
     Calculating Ownership, Counties, and Regions...
       enrich step 6/16 spatial join ownership
       enrich step 7/16 spatial join regions
       enrich step 8/16 spatial join veg
       enrich step 9/16 calculate ownership
       enrich